## Objectives

In this part we will:

- Develop an End to End ETL process with a log file

- Extract bank and market cap data from the JSON file `bank_market_cap.json` that we got from scraping a wikipedia webpage: 
https://web.archive.org/web/20200318083015/https://en.wikipedia.org/wiki/List_of_largest_banks

- Transform the market cap currency using the exchange rate data from the csv file `exchange_rates.csv` that we got from using ExchangeRate-API: https://exchangeratesapi.io/

- Load the transformed data into a seperate CSV


In [1]:
import glob
import pandas as pd
from datetime import datetime

In [2]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [3]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

## Extract Function

Extract function finds JSON files in the current working directory and calls the function extract_from_json to extract data from them and Storing these data in a Pandas Dataframe.


In [4]:
def extract():
    columns=['Name','Market Cap (US$ Billion)']
    data_frame = pd.DataFrame()
    for json_file in glob.glob('*.json'):
        data_frame = data_frame.append(extract_from_json(json_file),ignore_index=True)
        data_frame = data_frame.drop(columns = ["Rank"])[1:] 
        # renaming columns 
        data_frame.columns = columns
    return data_frame

In [5]:
#testing extract function
extract()

,Name,Market Cap (US$ Billion)
1,JPMorgan Chase,390.934
2,Industrial and Commercial Bank of China,345.214
3,Bank of America,325.331
4,Wells Fargo,308.013
5,China Construction Bank,257.399
...,...,...
66,Ping An Bank,37.993
67,Standard Chartered,37.319
68,United Overseas Bank,35.128
69,QNB Group,33.560


In [6]:
#get_specific_rate function return a specific exchange rate
def get_aspecific_rate(rate):
    df = extract_from_csv("exchange_rates.csv")
    df.columns = ["currency", 'rates']
    return(float(df[df['currency'] == "{}".format(rate)]['rates']))


In [7]:
# testing get_aspecific_rate function
get_aspecific_rate('GBP')

0.854124

## Transform

Uses extracted data and the specific exchange rate to preform the following Transformation

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`

In [8]:
def transform(extracted_data,rate):
    transformed_df = pd.DataFrame(extracted_data)
    transformed_df['Market Cap (GBP$ Billion)'] = round(transformed_df['Market Cap (US$ Billion)'].astype(float)*rate,3)
    return transformed_df.drop(columns=['Market Cap (US$ Billion)'])

In [9]:
#loading data into csv file named `bank_market_cap_gbp.csv`
def load(data_to_be_Loaded):
    # Write your code here
    data_to_be_Loaded = pd.DataFrame(data_to_be_Loaded)
    data_to_be_Loaded.to_csv("bank_market_cap_gbp.csv",index=False)

In [10]:
# log is a logging function that  
def log(message):
    # Write your code here
    time_now = datetime.now()
    with open('logs.txt','a') as f:
        f.write("{} ===> {} \n".format(time_now ,message))

# Running the ETL Process

In [11]:
log("ETL Job Started")
log( "Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
log("Transform phase Started")
transformed_data = transform(extracted_data,get_aspecific_rate('GBP'))
log("Transform phase Ended")
log("Load phase Started")
load(transformed_data)
log("Load phase Ended")
